# Vectara-Skunk-Client Example
In the code below we show how easy it is to get started with the client. Before these steps, please ensure you have:
1. Generate either an API Key or OAuth2 App from within Vectara's console.
2. Put these into a configuration ".vec_auth.yaml" in your home directory

Format for configuration should should match the following:
```yaml
default:
  customer_id : "1999999999"
  auth:
      # For API Key, you only need the API key
      api_key : "abcdabcdabcdabcdabcdabcdababcdabcd"
admin:
  customer_id : "1999999999" # Customer Id as a string
  # For OAuth2, you need app_client_id, app_client_secret, auth_url
  auth:
      app_client_id : "abcdabcdabcdabcdabcdabcdab"
      app_client_secret : "abcdabcdabcdabcdabcdabcdababcdabcdabcdabcdabcdabcdab"
```

In [2]:
%pip install -q vectara-skunk-client==0.4.11

Note: you may need to restart the kernel to use updated packages.


## Setup Logging
The Client has extensive logging but we need to make sure it's activated within our Python environment

In [3]:
import logging

logging.basicConfig(format='%(asctime)s %(name)-20s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S %z')

## Create the Client Factory
In the code below we create the client factory without any arguments, this will then use the configuration file in the users home directory.

The full factory flow is shown here:

<img src="https://github.com/davidglevy/vectara-skunk-client/raw/main/resources/images/factory-build-flow.png" alt="Factory Build Flow" width="600px"/>

In [6]:
from vectara.client.core import Factory

client = Factory().build()

08:53:18 +1100 Factory              INFO:initializing builder
08:53:18 +1100 Factory              INFO:Factory will load configuration from home directory
08:53:18 +1100 HomeConfigLoader     INFO:Loading configuration from users home directory [C:\Users\david]
08:53:18 +1100 HomeConfigLoader     INFO:Loading default configuration [default]
08:53:18 +1100 HomeConfigLoader     INFO:Parsing config
08:53:18 +1100 root                 INFO:We are processing authentication type [OAuth2]
08:53:18 +1100 OAuthUtil            INFO:Using provided OAuth2 URL [https://vectara-prod-1623270172.auth.us-west-2.amazoncognito.com/oauth2/token]
08:53:18 +1100 OAuthUtil            INFO:OAuth2 URL is [https://vectara-prod-1623270172.auth.us-west-2.amazoncognito.com/oauth2/token]
08:53:18 +1100 root                 INFO:initializing Client


In [7]:
# Quick test to verify we can see all corpora
admin_service = client.admin_service
corpora = admin_service.list_corpora()
for corpus in corpora:
    logging.info(f"Found corpus [{corpus.name}] with id [{corpus.id}]")

08:53:23 +1100 OAuthUtil            INFO:Current timestamp 2024-01-13 08:53:23.344876
08:53:23 +1100 OAuthUtil            INFO:First time requesting token, authenticating
08:53:24 +1100 OAuthUtil            INFO:Received OAuth token, will expire [01/13/2024, 09:53:24]
08:53:24 +1100 OAuthUtil            INFO:Already authenticated with non-expired token, expiry is [1705100004]
08:53:24 +1100 RequestUtil          INFO:URL for operation list-corpora is: https://api.vectara.io/v1/list-corpora
08:53:26 +1100 root                 INFO:Found corpus [Australia Broadband] with id [6]
08:53:26 +1100 root                 INFO:Found corpus [Australian Importation Laws] with id [10]
08:53:26 +1100 root                 INFO:Found corpus [High Court of Australia] with id [17]
08:53:26 +1100 root                 INFO:Found corpus [Public Reports] with id [8]
08:53:26 +1100 root                 INFO:Found corpus [SE] with id [7]
08:53:26 +1100 root                 INFO:Found corpus [South Australian St

## Check for Existing Corpus
We'll now check if our test corpus exists, and if so, delete it so we start with a clean slate.

In [8]:
admin_service = client.admin_service

corpora = admin_service.list_corpora("01-first-client-example")

if len(corpora) >= 1:
    for corpus in corpora:
        admin_service.delete_corpus(corpus.id)
else:
    logging.info("No existing corpus with the name client-example")

08:53:28 +1100 OAuthUtil            INFO:Current timestamp 2024-01-13 08:53:28.475589
08:53:28 +1100 OAuthUtil            INFO:Expiry            2024-01-13 09:53:24
08:53:28 +1100 OAuthUtil            INFO:Already authenticated with non-expired token, expiry is [1705100004]
08:53:28 +1100 RequestUtil          INFO:URL for operation list-corpora is: https://api.vectara.io/v1/list-corpora
08:53:29 +1100 root                 INFO:No existing corpus with the name client-example


## Create New Corpus
We now use the simple signature vectara.admin.AdminService#create_corpus

In [9]:
create_corpus_result = admin_service.create_corpus("01-first-client-example", description="Example Corpus for use from Jupyter")
logging.info(f"New corpus created {create_corpus_result}")
corpus_id = create_corpus_result.corpusId

08:53:31 +1100 OAuthUtil            INFO:Current timestamp 2024-01-13 08:53:31.509449
08:53:31 +1100 OAuthUtil            INFO:Expiry            2024-01-13 09:53:24
08:53:31 +1100 OAuthUtil            INFO:Already authenticated with non-expired token, expiry is [1705100004]
08:53:31 +1100 RequestUtil          INFO:URL for operation create-corpus is: https://api.vectara.io/v1/create-corpus
08:53:33 +1100 AdminService         INFO:Created new corpus with 242
08:53:33 +1100 root                 INFO:New corpus created CreateCorpusResponse(corpusId=242, status=Status(code=<StatusCode.OK: 0>, statusDetail='Corpus Created', cause=None))


## Load the Corpus
We'll now load the corpus using a file on our computer. This file is a word document which will be automatically parsed by Vectara

In [10]:
indexer_service = client.indexer_service
indexer_service.upload(corpus_id, "C:\\Users\\david\\OneDrive\\Documents\\Publications\\RAG Options.docx")

08:53:33 +1100 IndexerService       INFO:Headers: {"c": "1623270172", "o": "242"}
08:53:33 +1100 OAuthUtil            INFO:Current timestamp 2024-01-13 08:53:33.728835
08:53:33 +1100 OAuthUtil            INFO:Expiry            2024-01-13 09:53:24
08:53:33 +1100 OAuthUtil            INFO:Already authenticated with non-expired token, expiry is [1705100004]
RAG Options.docx: 695kB [00:04, 163kB/s]                                                                             


UploadDocumentResponse(response=UploadDocumentResponseInner(status=None, quotaConsumed=StorageQuota(numChars='7936', numMetadataChars='3592')), document=None)

## Query the Corpus
Lets run a basic query on the corpus with only a single document in it - answers improve with more relevant material but this shows the complete loop.

In [11]:
query_service = client.query_service
query = "Why is Vectara a good RAG system?"
response = query_service.query(query, corpus_id)


08:53:40 +1100 OAuthUtil            INFO:Current timestamp 2024-01-13 08:53:40.706530
08:53:40 +1100 OAuthUtil            INFO:Expiry            2024-01-13 09:53:24
08:53:40 +1100 OAuthUtil            INFO:Already authenticated with non-expired token, expiry is [1705100004]
08:53:40 +1100 RequestUtil          INFO:URL for operation query is: https://api.vectara.io/v1/query


## Show Result in Markdown
We'll now use the utility method "renderMarkdown" to render the Python dict

In [12]:
from vectara.client.util import render_markdown
from IPython.display import display, Markdown
rendered = render_markdown(query, response)
display(Markdown(rendered))


# Query: Why is Vectara a good RAG system?

Vectara is a good Retrieval Augmented Generation (RAG) system because it is designed specifically to cater to "SaaS for serving RAG operationally," making it a ready-to-use system for these workloads [1]. Developed with a focus on application builders, it provides an easy path to Generative AI solutions [5]. Vectara's RAG features are considered to be on another level and it aims to be the "Snowflake of RAG," ensuring an optimal level of abstraction for developers coming from different platforms [3]. It was established when RAG was a less explored space, so it has a first-mover advantage in this field [2]. Lastly, its advanced features such as Access Control Lists (ACLs) and multi-corpus searches set it apart from other RAG systems [4].

 1. · Multi-Corpus Queries? · ACLs on the Corpus? <b>Scoring Category Through the lens of “SaaS specifically for serving RAG operationally”, Vectara is the only ready-to-go system for serving these workloads.</b> The rest are either “platforms that can be used for” or “libraries” rather than a capability. No surprise, Vectara scored highest here. *score: 0.93919635, doc-id: RAG Options.docx*
 2. RAG Options What the thing is: Choosing a RAG in today’s market Options · Cohere · OpenAI · Azure Search · Google AI · LangChain · Llama Index · Databricks Summarise · Background Sentence · Who each tech is created for (App Builders | Data Engineers | DSs) · Cost model (scale to zero) · Abstraction Strengths/Weaknesses · Ease of Use (right abstraction etc) · Operational Cost · Trust · Model Coupling · Advanced Features (Hybrid) Conclusion Introduction Wow what a difference a day makes!! <b>When Vectara was founded the Retrieval Augmented Generation (RAG) was a very lonely space.</b> But from the explosion of LLMs in 2022 have also seen the number of solutions which propose a RAG element to also increase. With this document, we aim to look at who are the major players in this space and do a feature comparison. *score: 0.844621, doc-id: RAG Options.docx*
 3. You could achieve Data Warehousing with Databricks in 2021 but there was a lot of banging, tears and a fair few sharp corners to get something equivalent to Snowflake. Databricks bridged that gap with Serverless and Unity Catalog, but kudos where kudos is warranted, Snowflake is still the optimum level of abstraction for many Data Warehouse folk coming from 20 years on legacy platforms. <b>Vectara’s goal is to be the Snowflake of RAG, being the equivalent for Application Builders.</b> Team Costs How many cooks does it take to get your RAG pipeline working … and to continue working. This measure is subjective but based on my experiences working with technical teams across my career. *score: 0.84389603, doc-id: RAG Options.docx*
 4. The Players Like the introduction of the tributes in “The Hunger Games” here are the contenders. Amr would have been great representing District 12!! <b>Vectara Founded in 2019 which has been designed from the ground up to be a focused RAG solution for application builders with easy APIs.</b> The “Snowflake” of the category – purpose built to allow organisations to think about their IP in terms of Corpus with advanced features like ACLs and multi-corpus searches. Cohere Also founded in 2019 with multiple researchers from “Google Brain”, one of which was the author of the famous “Attention is All you Need” paper which introduced the transformer architecture and lay the groundwork for other heavy weights below. *score: 0.83304876, doc-id: RAG Options.docx*
 5. Fifth: Azure Search (21.2) Conclusion I mean, we’re tooting our own horn and came out first? Who would have thought. <b>But seriously though: Vectara’s RAG features are on another level and the focused delivery of a platform for application builders by application builders has resulted in an incredibly easy path to Generative AI solutions.</b> image1.png *score: 0.8089236, doc-id: RAG Options.docx*
 6. Not what I intended at all. https://twitter.com/elonmusk/status/1626516035863212034 All that being said, they are the leaders in terms of General LLMs and have made moves towards RAG with their Assistant feature. <b>Azure Search This is a “build it yourself” RAG system – achievable but you own the most of the steps and leveraging Open AI’s features to their Cognitive Search.</b> I mean, at least they didn’t completely rip off one of their Open Source partners tech stacks and rebadge it as their own this time. Google AI TBD LangChain TBD Llama Index TBD Databricks I would be remiss not to also mention Databricks as a former employer. *score: 0.7482908, doc-id: RAG Options.docx*
 7. No surprise, Vectara scored highest here. Completeness In the realm of “Don’t make me think”, how much of each platform is done for our users? <b>Again Vectara is platform that “just works” whereas others require many of the additional steps to be both done and wired in.</b> Again, Vecata scores well here. For example, Cohere is mature but you have to “do-it-yourself” to accomplish the major parts of RAG (parsing, encoding, vector storage and search): https://docs.cohere.com/docs/retrieval-augmented-generation-rag#step-2-fetching-relevant-documents Vendor	Comment by David Levy: Justin: Could use your expertise with Google AI, LangChain and LlamaIndex Parsing Encoding Vector Storage Retrieval Prompt Engineering LLM Execution Score	Comment by David Levy: I have a sheet to calculate scores - will update after we put in Google AI, LangChain and LlamaIndex Vectara Yes Yes Yes Yes Yes Yes 10.0 Cohere No No No No Yes Yes 3.3 OpenAI Yes Yes Yes Partial Yes Yes 9.2 Azure Search* No No Yes Partial Partial Partial 4.2 Google AI LangChain Llama Index Databricks No No Yes Partial Partial Partial 4.2 Note: Partial scores relate either to situations where you “need to build/maintain it” or “you can by integrating other parts of the stack, e.g. “Azure Search + Cosmos”. *score: 0.74707174, doc-id: RAG Options.docx*
 8. Trust The scoring metric here was applied in relation to how much you trust the Vendor to provide reliable responses for both the retrieval and summarisation phases of the RAG pipeline. If the onus is entirely on the client to perform their own evaluation, this scores poorly amongst “application builders” who just want something that works … and works well. <b>Though Vectara scored well (I mean, yes we’re a bit biased) we still didn’t put a 10 as the market vertical is still too new – maybe after another year of great results and expanded use of Boomerang we’ll self-report a 9.</b> Advanced RAG Features What additional features doe the vendor support specifically for RAG? Result Summary Vendor Category Completeness Abstraction Team Costs Trust Adv Features Vectara SaaS (10) 10.0 Corpus (10) Low (10) 8 10 Cohere PaaS(5) 3.3 Low Level (6) Mid (5) 3 1 OpenAI SaaS(10) 9.2 Assistant (7) Mid (5) 6 3 Azure Search PaaS(5) 4.2 3 High (2) 5 2 Google AI LangChain Llama Index Databricks PaaS(5) 4.2 Low Level (4) High (2) 5 3 Databricks does a little better here with lineage from Unity Catalog lineage and Model Monitoring – two of the best features I was sad to leave behind. *score: 0.74091953, doc-id: RAG Options.docx*
 9. <b>RAG Options What the thing is: Choosing a RAG in today’s market Options · Cohere · OpenAI · Azure Search · Google AI · LangChain · Llama Index · Databricks Summarise · Background Sentence · Who each tech is created for (App Builders | Data Engineers | DSs) · Cost model (scale to zero) · Abstraction Strengths/Weaknesses · Ease of Use (right abstraction etc) · Operational Cost · Trust · Model Coupling · Advanced Features (Hybrid) Conclusion Introduction Wow what a difference a day makes!!</b> When Vectara was founded the Retrieval Augmented Generation (RAG) was a very lonely space. But from the explosion of LLMs in 2022 have also seen the number of solutions which propose a RAG element to also increase. *score: 0.71499735, doc-id: RAG Options.docx*
 10. Open AI, whilst it has an assistant, is not aligned to a corpus abstraction and lacks expected maturity which also affects other categories. Final Scores (recalculate with missing): 1. <b>First: Vectara (58) 2.</b> Fourth: Databricks (23.2) 5. Fifth: Azure Search (21.2) Conclusion I mean, we’re tooting our own horn and came out first? *score: 0.6951151, doc-id: RAG Options.docx*
